# Holoviz and hvPlot

Python's visualization landscape is complex and confusing!

<div style="clear:left;">
<img src="landscape.png" width=75% align="left" style="margin: 0px 30px">
</div>

HoloViz to the rescue!!

## HoloViz libraries

<img width="800" src="pn_hp_hv_gv_ds_lu_pa_cs.png"/>

-   [Panel](http://panel.pyviz.org): Assembling objects from many different libraries into a layout or app, whether in a Jupyter notebook or in a standalone servable dashboard
-   [hvPlot](http://hvplot.pyviz.org): Quickly return Bokeh/Matplotlib/Plotly-based HoloViews plots from Pandas, Xarray, orother data structures
-   [HoloViews](http://holoviews.org): Declarative objects for instantly visualizable data, building Bokeh/Matplotlib/Plotly plots from convenient high-level specifications
-   [GeoViews](http://geoviews.org): Visualizable geographic data that that can be mixed and matched with HoloViews objects
-   [Datashader](http://datashader.org): Rasterizing huge datasets quickly as fixed-size arrays or images
-   [Lumen](https://lumen.holoviz.org): Building Panel dashboards with HoloViews plots from a text file (low code) or GUI (no code)
-   [Param](http://param.pyviz.org): Declaring user-relevant parameters, making it simple to work with widgets inside and outside of a notebook context
-   [Colorcet](http://colorcet.pyviz.org): Perceptually accurate continuous and categorical colormaps for any viz tool


These libraries work with and are built upon many other familiar open source [libraries](https://holoviz.org/tutorial/Overview.html).

### Start with hvPlot
hvPlot is a unifying library that easily connects your data (e.g. Pandas, Xarray, etc.) to the rest of the HoloViz world.

<img width="800" src="diagram.svg"/>


## hvPlot and Pandas

In [ ]:
import pandas as pd

In [ ]:
datafile = '/home/jovyan/.cache/noaa-data/data.txt'
df = pd.read_csv(datafile, sep='\s+',
                 na_values=[-9999.0, -99.0],
                 parse_dates=[1])

In [ ]:
df.rename(columns={'LST_DATE': 'Date'}, inplace=True)
df.set_index('Date', inplace=True)

In [ ]:
df.head()

In [ ]:
df_mm = df.resample('MS').mean(numeric_only=True)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df.T_DAILY_MEAN.plot()

In [ ]:
import hvplot.pandas
df.T_DAILY_MEAN.hvplot()

In [ ]:
df.T_DAILY_MEAN.hvplot.scatter()

In [ ]:
df.T_DAILY_MEAN.hvplot.scatter(s=300, alpha=0.1)

In [ ]:
hvplot.help('scatter')

### Compositing

In [ ]:
df.T_DAILY_MEAN.hvplot.scatter(s=300, alpha=0.1, width=450, height=300) + \
df_mm.T_DAILY_MEAN.hvplot(width=450, height=300)

In [ ]:
df.T_DAILY_MEAN.hvplot.scatter(s=300, alpha=0.1, legend=False) * \
df_mm.T_DAILY_MEAN.hvplot(legend=False) * \
df_mm.T_DAILY_MEAN.hvplot.scatter(c='m', s=100, legend=False)

In [ ]:
df.T_DAILY_MEAN.hvplot.scatter(s=300, alpha=0.1, label='Daily Mean T', width=800, height=400).opts(legend_position='top_left')  * \
df_mm.T_DAILY_MEAN.hvplot(label='Monthly Mean T') * \
df_mm.T_DAILY_MEAN.hvplot.scatter(c='m', s=50, label='Monthly Mean T')

In [ ]:
df.T_DAILY_AVG.hvplot.hist(bins=50)

### Earthquake Catalog

In [ ]:
eqs = pd.read_csv('http://www.ldeo.columbia.edu/~rpa/usgs_earthquakes_2014.csv',
                   index_col='time')

In [ ]:
eqs.tail()

In [ ]:
len(eqs)

In [ ]:
eqs.hvplot.scatter(x='longitude', y='latitude')

In [ ]:
eqs.hvplot.hexbin(x='longitude', y='latitude', cnorm='eq_hist')

In [ ]:
eqs.hvplot.scatter(x='longitude', y='latitude', s=0.008*eqs['mag'].values**6, c=eqs['mag'].values, cmap='jet')

In [ ]:
eqs.hvplot.points(x='longitude', y='latitude', rasterize=True, colorbar=False,
                  cnorm='eq_hist', geo=True, coastline='50m', frame_width=500)

In [ ]:
eqs.hvplot.points(x='longitude', y='latitude', rasterize=True, dynspread=False, cnorm='eq_hist',
                  geo=True, coastline='50m', projection='Robinson')

In [ ]:
points = pd.DataFrame(np.random.multivariate_normal((0,0), [[0.1, 0.1], [0.1, 1.0]], (int(4e5),)),
                     columns=('x', 'y'))

In [ ]:
points.hvplot.scatter(x='x', y='y')

In [ ]:
points.hvplot.scatter(x='x', y='y', rasterize=True)

In [ ]:
points.hvplot.scatter(x='x', y='y', rasterize=True, dynspread=False) + points.hvplot.scatter(x='x', y='y', rasterize=True, dynspread=True)

## hvPlot and Xarray

In [ ]:
import numpy as np
import xarray as xr
import hvplot.xarray

In [ ]:
url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
ds = xr.open_dataset(url, drop_variables=['time_bnds'])
ds = ds.sel(time=slice('1960', '2018')).load()
ds

In [ ]:
ds.sst.hvplot()

In [ ]:
ds.sst[0].hvplot.contour(levels=np.arange(-4,31), cmap='viridis', clim=(0, 31))

In [ ]:
ds.sst.hvplot.contourf(levels=np.arange(-4,31), cmap='viridis', clim=(0, 31))

In [ ]:
(ds.groupby('time.year').mean()).hvplot.contourf(x='lon',y='lat', levels=np.arange(-4,31), cmap='viridis', clim=(0, 31))

In [ ]:
import cartopy.crs as ccrs

In [ ]:
ccrs.InterruptedGoodeHomolosine

In [ ]:
ds.sst.hvplot.quadmesh(
    'lon', 'lat', 'sst', projection=ccrs.Robinson(),
    global_extent=True, cmap='viridis',
    coastline=True, frame_width=700
)

In [ ]:
air_ds = xr.tutorial.open_dataset('air_temperature').load()

air_ds.hvplot.quadmesh(
    'lon', 'lat', 'air', projection=ccrs.Orthographic(-90, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)